# MuseScore Piano Pieces Scraping

## Format of MuseScore:

ex: https://musescore.com/user/17629331/scores/5161444
### Artist name:
    <meta property="musescore:composer" content="Christopher Negron">
### Midi file:
     <h5><a href="/score/5161444/download/mid" rel="nofollow">MIDI</a></h5>
     <p>Open in editors and sequencers</p>
### Detailed information:
    <div class="read-more">
        <h4>Uploaded on Jul 12, 2018</h4>
        <div class="description ">
            <div class="read-more-inner">
                <div></div>
                                <p>This song was made from the depths of my soul: It&#039;s that feeling you get when you&#039;re in love with someone and you just want to watch the stars with them. Well, this song represents looking up at the stars with the person you love, so please enjoy Stargaze Ballade!</p>
                            <div class="more-info">

                    <table data-striping="1">
                        <tbody>
                            <tr>
                                <td>Pages</td>
                                <td>10</td>
                            </tr>
                            <tr>
                                <td>Duration</td>
                                <td>06:43</td>
                            </tr>
                            <tr>
                                <td>Measures</td>
                                <td>203</td>
                            </tr>
                            <tr>
                                <td>Key signature</td>
                                <td>natural</td>
                            </tr>
                            <tr>
                                <td>Parts</td>
                                <td>1</td>
                            </tr>
                            <tr>
                                <td>Part names</td>
                                <td>Piano</td>
                            </tr>
                            <tr>
                                <td>Privacy</td>
                                <td><i class="icon-public"></i> Everyone can see this score</td>
                            </tr>
                            <tr>
                                <td>License</td>
                                <td><i class="icon-copyright"></i> None (All rights reserved)</td>
                            </tr>
                        </tbody>
                    </table>
                </div>
            </div>
        </div>
        <a href="#" class="read-more-toggle">Show more</a>
        <a href="#" class="read-less-toggle">Show less</a>
    </div>


# Make a list of links to scrape
### (ex: https://musescore.com/user/17629331/scores/5161444)

## Browsing page with list of pieces: https://musescore.com/hub/piano
![Inspect Element](screen.png)


In [ ]:
from lxml import html
import urllib
import requests
import shutil

In [ ]:
links = []
url = "https://musescore.com/hub/piano"
page = requests.get(url)
tree = html.fromstring(page.content)
print(tree.xpath("//title/text()"))

In [14]:
links = tree.xpath("//article[@class='score-card']//h2//a[@rel='bookmark']/@href")

In [15]:
links[:5]

['/user/1885016/scores/5162398',
 '/user/29089604/scores/5162396',
 '/user/14651976/scores/5162393',
 '/user/28274737/scores/5162382',
 '/user/28015414/scores/5162381']

## But that's actually just page one and there are pages 2,3,4,5... 100!
### So let's make a more general format:

In [16]:
url_default = "https://musescore.com/hub/piano?page=" # + the page number

In [ ]:
links = []
for page in range(1,10):
    page = requests.get(url_default+str(page)) # https://musescore.com/hub/piano?page=1,2,3,4,5,...,10
    tree = html.fromstring(page.content)
    this_page_links = tree.xpath("//a[@rel='bookmark']/@href")
    
    count = 0
    for link in this_page_links:
        link_new = "https://musescore.com" + link
        this_page_links[count] = link_new
        count += 1
    links.extend(this_page_links)
print(links[:100])

# Scrape one link

In [18]:
scrape_this = links[0]
scrape_this

'https://musescore.com/user/1885016/scores/5162398'

In [19]:
page = requests.get(scrape_this)
tree = html.fromstring(page.content)

## Download Midi File

In [20]:
midi_link = tree.xpath("//a[contains(text(), 'MIDI')]/@href")
midi_link = "https://musescore.com" + midi_link[0]
midi_link

'https://musescore.com/score/5162398/download/mid'

### You need to be logged in to download the midi file.

In [43]:
payload = {
    "name": "cwkeam", 
    "pass": "12345678", 
    "form_build_id": "<CSRF_TOKEN>"
}
session_requests = requests.session()

login_url = "https://musescore.com/user/login?q=user/login"
result = session_requests.get(login_url)
tree = html.fromstring(result.text)
authenticity_token = list(set(tree.xpath("//input[@name='form_build_id']/@value")))[0]
print(authenticity_token)
payload['form_build_id'] = authenticity_token
print(payload)

result = session_requests.post(
    login_url, 
    data = payload, 
    headers = dict(referer=login_url)
)

print(result.status_code)  
print(result.headers['content-type'])  

with open('./res.html', 'wb') as f:  
     f.write(result.content)

form-FBscnlYrkKrCyOEYG6KnHu3J9yEaQ6azpxVsHue90p8
{'name': 'cwkeam', 'pass': '12345678', 'form_build_id': 'form-FBscnlYrkKrCyOEYG6KnHu3J9yEaQ6azpxVsHue90p8'}
200
text/html; charset=UTF-8


In [44]:
# Test if it worked
## If it worked, it should return nothing.
print(midi_link)
url = midi_link
result = session_requests.get(
    url, 
    headers = dict(referer = url)
)

https://musescore.com/score/5162398/download/mid


In [45]:

print(result.status_code)  
print(result.headers['content-type'])  
print(result.encoding) 


with open('./c.mid', 'wb') as f:  
    f.write(r_mid.content)

200
text/html; charset=UTF-8
UTF-8
